#### This is a sandbox playground for manual golfing that combines elements from several pubicly shared notebooks. This is meant to run in interactive mode.

Version 1: Start with the current highest scoring public notebook: https://www.kaggle.com/code/tonylica/400-task-with-smart-solution-search-verification

Minor improvements were made to Task 285 for an improved score of 905145. There is much opportunity to reduce the code length in this puzzle.

Version 2: Retrieve improved version from this notebook and further improved Task 287 for total score of 905155.

Version 3: Significantly reduced original code size of Task 5 from 758 to 677 using Co-pilot, following the strategy of V2, and then further compressed using ChatGPT. However, the compressed length only improved by 10 for total score of 905165.

Version 4: A new high scoring notebook has been uploaded - https://www.kaggle.com/code/tonylica/community-baselines-my-arc-10-pack-2-10

## Retrieval of solutions and preliminaries

In [ ]:
!mkdir submission

# Version 1
#!cp /kaggle/input/400-task-with-smart-solution-search-verification/submission/*.py /kaggle/working/submission

# Version 2&3 (don't forget to check for update to get latest codes)
# !cp /kaggle/input/gcgc-playground/submission/*.py /kaggle/working/submission

# Version 4
!cp /kaggle/input/community-baselines-my-arc-10-pack-3-10/submission/*.py /kaggle/working/submission

In [ ]:
import warnings
warnings.filterwarnings("ignore")

!pip install zopfli

In [ ]:
import multiprocessing
import time

def run_with_timeout(func, args=(), kwargs={}, timeout=5):
    def wrapper(queue):
        try:
            result = func(*args, **kwargs)
            queue.put(result)
        except Exception as e:
            queue.put(e)

    queue = multiprocessing.Queue()
    p = multiprocessing.Process(target=wrapper, args=(queue,))
    p.start()
    p.join(timeout)
    
    if p.is_alive():
        p.terminate()
        p.join()
        raise TimeoutError(f"Function timed out after {timeout} seconds")
    
    result = queue.get()
    if isinstance(result, Exception):
        raise result
    return result

In [ ]:
import re
import ast

default_code = '''
def p(input):
    return 
'''

def update(task_num):
    file_num = f"{task_num:03}"
    filename = f"task{file_num}.py"

    with open('task.py', "r", encoding="utf-8") as f:
        code = f.read()
        
    print(f'Task num: {task_num} | File num: {file_num}')
    examples = load_examples(task_num)
    
    try:
        run_with_timeout(verify_program, args=(task_num, examples), timeout=10)  # timeout 10s
        with open('./submission/' + filename, "w", encoding="utf-8") as f:
            f.write(code)
    except TimeoutError:
        print("TIMEOUT")
        with open('./submission/' + filename, "w", encoding="utf-8") as f:
            f.write(default_code)
        print("="*60)
    except Exception as e:
        print(f"ERROR: {e}")
        with open('./submission/' + filename, "w", encoding="utf-8") as f:
            f.write(default_code)
        print("="*60)
    print(f'File {file_num} updated')
    print("="*60)

In [ ]:
import sys
sys.path.append("/kaggle/input/google-code-golf-2025/code_golf_utils")
from code_golf_utils import *

from tqdm import tqdm
ret = []
for task_num in tqdm(range(1, 401)):
    text = ""
    examples = load_examples(task_num)
    total_samples = len(examples['train']) + len(examples['test'])
    examples['train'] += examples['test']
    for i in range(total_samples):
        text += f'\nExample {i + 1}:\n - Input:\n'
        text += '],\n '.join(str(examples['train'][i]['input']).split('], '))
        text += '\n - Output:\n'
        text += '],\n '.join(str(examples['train'][i]['output']).split('], '))
        text += '\n'
    ret.append(text)
print(len(ret))

In [ ]:
import re
import ast
import zlib

def extract_and_decompress(filename):
    with open(filename, 'r', encoding='latin1') as f:
        lines = f.readlines()

    # Remove encoding comment and import if present
    content = ''.join(lines[2:]) if lines[0].startswith('#encoding:') else ''.join(lines)

    # Flexible pattern: match zlib.decompress(bytes('...', 'L1')) or similar
    pattern = r"""
        exec\s*\(\s*zlib\.decompress\s*\(\s*     # exec(zlib.decompress(
        bytes\s*\(\s*                            # bytes(
        (?P<quote>['"]{1,3})                     # opening quote (1 to 3 of ' or ")
        (?P<data>.*?)                            # the actual string data
        (?P=quote)\s*,\s*                        # matching closing quote
        ['"](?i:l1|latin-1)['"]\s*               # encoding string (L1 or latin-1, case-insensitive)
        \)\s*\)\s*\)                             # closing ))
    """

    match = re.search(pattern, content, re.DOTALL | re.VERBOSE)
    if not match:
#        print("❌ No matching compressed payload found.")
        print(content)
        return None

    str_literal = match.group('quote') + match.group('data') + match.group('quote')

    try:
        # Turn the string literal into a Python string (handles escapes)
        raw_str = ast.literal_eval(str_literal)
        raw_bytes = raw_str.encode('latin1')
        decompressed = zlib.decompress(raw_bytes)
        return decompressed.decode('utf-8')
    except Exception as e:
        print(f"❌ Failed to decompress: {e}")
        return None


In [ ]:
#taylorsamarel/qwen2-5-32b-arc-local-score-32-solved-script
import zipfile, json, os, copy, json

def check(solution, task_num, valall=False):
    if task_num == 157: return True # this one just takes a while to run
    task_data = load_examples(task_num)
    #print(task_num, max(1, 2500 - len(solution.encode('utf-8'))))
    try:
        namespace = {}
        exec(solution, namespace)
        if 'p' not in namespace: return False
        all_examples = task_data['train'] + task_data['test'] + task_data['arc-gen']
        examples_to_check = all_examples if valall else all_examples[:3]
        for example in examples_to_check:
            input_grid = copy.deepcopy(example['input'])
            expected = example['output']
            try:
                actual = namespace['p'](input_grid)
                actual = [[int(x) if int(x) == x else x for x in row] for row in actual]
                if json.dumps(actual) != json.dumps(expected):
                    return False
            except:
                return False
        return True
    except Exception as e:
        print(e)
        return False

## Select the task number and display puzzle and source code

In [ ]:
# Select puzzle and display
task_num = 37

# Extracts the source code and display as output. The output code may then be copied and pasted into the following cell.
filename = f'/kaggle/working/submission/task{str(task_num).zfill(3)}.py'

source_code = extract_and_decompress(filename)

if source_code:
    print(source_code)
    
examples = load_examples(task_num)
show_examples(examples['train'] + examples['test'])

# Display color legend
show_legend()


In [ ]:
%%writefile task.py
def p(e,t=range):
 u={};e=eval(str(e))
 for n,r in enumerate(e):
  for p,i in enumerate(r):
   if i:u.setdefault(i,[]).append((n,p))
 for i in u:
  (n,p),(r,d)=u[i];a,d=(r>n)-(r<n),(d>p)-(d<p)
  for s in t(abs(r-n)+1):e[n+s*a][p+s*d]=i
 return e

In [ ]:
# original code
verify_program(task_num, examples)


In [ ]:
%%writefile task.py
def p(e,t=range):
 u={};e=eval(str(e))
 for n,r in enumerate(e):
  for p,i in enumerate(r):
   if i:u.setdefault(i,[]).append((n,p))
 for i in u:
  (n,p),(r,d)=u[i];a,d=(r>n)-(r<n),(d>p)-(d<p)
  for s in t(abs(r-n)+1):e[n+s*a][p+s*d]=i
 return e

In [ ]:
# Asked co-pilot to compress the code using a similar example to Task 287 in V2
verify_program(task_num, examples)


In [ ]:
%%writefile task.py
def p(e,t=range):
 u={};e=eval(str(e))
 for n,r in enumerate(e):
  for p,i in enumerate(r):
   if i:u.setdefault(i,[]).append((n,p))
 for i in u:
  (n,p),(r,d)=u[i];a,d=(r>n)-(r<n),(d>p)-(d<p)
  for s in t(abs(r-n)+1):e[n+s*a][p+s*d]=i
 return e

In [ ]:
# Further reduction of above using ChatGPT
verify_program(task_num, examples)


In [ ]:
# Update the version to be submitted
update(task_num)

In [ ]:
from zipfile import ZipFile
import zipfile
import zopfli.zlib
import zlib
import warnings

def zip_src(src_code):
    candidates=[src_code]
    for compress in[zopfli.zlib.compress,lambda d:zlib.compress(d,9)]:
        for trailing in[b'',b'\n']:
            src=src_code+trailing
            while(comp:=compress(src))[-1]==ord('"'):src+=b'#'
            for delim in[b"'",b'"']:
                esc_map={0:b'\\x00',ord('\n'):b'\\n',ord('\r'):b'\\r',ord('\\'):b'\\\\',delim[0]:b'\\'+delim}
                sanitized=b''.join(esc_map.get(b,bytes([b]))for b in comp)
                compressed=b'import zlib\nexec(zlib.decompress(bytes('+delim+sanitized+delim+b',"L1")))'
                if max(sanitized)>127:compressed=b'#coding:L1\n'+compressed
                else:print('no header needed!')
                candidates.append(compressed)
            esc_map={0:b'\\x00',ord('\r'):b'\\r',ord('\\'):b'\\\\'}
            sanitized=b''.join(esc_map.get(b,bytes([b]))for b in comp)
            compressed=b'import zlib\nexec(zlib.decompress(bytes("""'+sanitized+b'""","L1")))'
            if max(sanitized)>127:compressed=b'#coding:L1\n'+compressed
            else:print('no header needed!')
            candidates.append(compressed)
    valid_options=[]
    for code in candidates:
        try:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=SyntaxWarning)
                with open('tmp.py','wb')as f:f.write(code)
                with open('tmp.py','rb')as f:x=f.read()
                exec(x,{})
                valid_options.append(code)
        except:0
    return min(valid_options,key=len)

files = {}
total_save=0

with ZipFile("submission.zip", "w", zipfile.ZIP_DEFLATED) as zipf:
    o=open('task.py','rb').read().strip()
    zipped_src = zip_src(o)
    files_len = min(len(o), len(zipped_src))
    improvement = len(o) - len(zipped_src)
    if improvement > 0:
        open('/kaggle/working/submission/task' + str(task_num).zfill(3) + '.py','wb').write(zipped_src)
    else:
        open('/kaggle/working/submission/task' + str(task_num).zfill(3) + '.py','wb').write(o)
    zipf.write('/kaggle/working/submission/task' + str(task_num).zfill(3) + '.py')
print("Compression Save: ", improvement)
print("Code length: ", len(zipped_src))

In [ ]:
# Generate submission.zip file for submission
files = {}
total_save=0
with ZipFile("submission.zip", "w", zipfile.ZIP_DEFLATED) as zipf:
    for f in range(1,401):
        try:
            o=open('/kaggle/working/submission/task' + str(f).zfill(3) + '.py','rb').read().strip()
            zipped_src = zip_src(o)
            files[f] = min(len(o), len(zipped_src))
        except:
            continue
        #https://www.kaggle.com/code/cheeseexports/big-zippa
        improvement = len(o) - len(zipped_src)
        if improvement > 0:
            total_save += improvement
            open('/kaggle/working/submission/task' + str(f).zfill(3) + '.py','wb').write(zipped_src)
        else:
            open('/kaggle/working/submission/task' + str(f).zfill(3) + '.py','wb').write(o)
        zipf.write('/kaggle/working/submission/task' + str(f).zfill(3) + '.py')
print("Submission file generated")

## Status of each task and final score of submission

In [ ]:
#https://docs.google.com/spreadsheets/u/1/d/e/2PACX-1vQ7RUqwrtwRD2EJbgMRrccAHkwUQZgFe2fsROCR1WV5LA1naxL0pU2grjQpcWC2HU3chdGwIOUpeuoK/pubhtml#gid=1427788625
top=[58, 90, 58, 80, 206, 51, 62, 84, 109, 68, 121, 127, 140, 70, 93, 43, 99, 323, 105, 146, 57, 91, 195, 62, 131, 52, 103, 63, 108, 94, 45, 39, 73, 125, 83, 75, 105, 51, 60, 69, 49, 139, 56, 255, 45, 170, 55, 92, 81, 85, 115, 40, 21, 280, 83, 40, 48, 103, 156, 48, 63, 143, 74, 152, 91, 268, 33, 116, 151, 78, 119, 54, 46, 79, 86, 276, 126, 60, 123, 253, 91, 50, 40, 62, 50, 172, 36, 101, 236, 159, 63, 86, 99, 102, 73, 325, 108, 88, 115, 85, 281, 150, 29, 84, 148, 67, 162, 56, 81, 85, 60, 109, 25, 64, 54, 20, 148, 271, 106, 97, 89, 82, 75, 96, 126, 54, 65, 61, 47, 65, 125, 86, 298, 168, 32, 105, 141, 104, 94, 36, 94, 40, 135, 53, 191, 58, 83, 141, 75, 30, 108, 40, 133, 99, 18, 146, 248, 269, 109, 105, 82, 96, 131, 32, 136, 61, 71, 111, 129, 196, 51, 20, 218, 97, 75, 64, 51, 47, 21, 79, 67, 169, 93, 100, 143, 60, 92, 61, 111, 109, 241, 110, 81, 67, 105, 112, 54, 122, 84, 84, 208, 102, 64, 93, 166, 144, 81, 215, 289, 20, 48, 105, 92, 62, 42, 114, 95, 56, 257, 87, 87, 103, 51, 171, 132, 139, 52, 119, 73, 114, 43, 58, 297, 118, 61, 54, 67, 223, 99, 99, 21, 54, 79, 64, 106, 105, 95, 72, 26, 118, 89, 57, 129, 84, 242, 95, 74, 61, 85, 135, 47, 39, 122, 216, 104, 102, 46, 239, 63, 117, 86, 89, 116, 71, 136, 38, 194, 118, 107, 179, 145, 83, 82, 220, 288, 109, 56, 89, 63, 67, 61, 56, 59, 70, 54, 63, 43, 55, 54, 87, 31, 89, 62, 92, 57, 71, 50, 226, 38, 78, 32, 44, 63, 96, 63, 71, 59, 54, 194, 68, 55, 48, 102, 259, 160, 30, 67, 163, 54, 134, 83, 58, 89, 66, 107, 93, 46, 64, 37, 119, 132, 111, 65, 78, 90, 58, 50, 94, 214, 91, 67, 84, 92, 96, 98, 105, 86, 97, 64, 45, 193, 69, 213, 155, 111, 365, 129, 138, 113, 260, 109, 48, 39, 112, 53, 30, 55, 143, 141, 27, 79, 134, 121, 62, 25, 52, 218, 61, 57, 99, 63, 149, 63, 91, 53, 179, 121, 77, 64, 67]

score = 0
for taskNo in files:
    try:
        solution = open('/kaggle/working/submission/task' + str(taskNo).zfill(3) + '.py','rb').read()
        if check(solution, taskNo, valall=True):
            s = max([0.1,2500-len(solution)])
            print(taskNo, 2500-s, top[taskNo-1], top[taskNo-1]-(2500-s))
            score += s
        else: print(taskNo, ":L")
    except: pass
print('Score:', score)

In [ ]:
# Done